In [1]:
import matplotlib.pyplot as plt
import numpy as np
import re
import os
import seaborn as sns
import pandas as pd

In [2]:
def parse_ping(txt_file, ignore_time = None):
    with open(txt_file, 'r') as f:
        txt = f.read()
    time_arr = []
    for line in txt.strip().split('\n'):
        time = None
        icmp_seq = 'icmp_seq='
        if (icmp_seq in line):
            temp1 = line[line.index(icmp_seq) + len(icmp_seq):]
            time = int(temp1[:temp1.index(' ')])
        if (ignore_time is not None and time is not None and time <= ignore_time):
            try:
                temp = float(line[line.index("time=") + 5:].replace(' ms', ''))
                time_arr += [temp]
            except:
                continue
    return np.array(time_arr)

def parse_server(txt_file):
    with open(txt_file, 'r') as f:
        txt = f.read()
    temp = re.sub(' +', ' ', txt.strip())
    string_index_retr = '[ ID] Interval Transfer Bitrate Retr'
    string_index = '[ ID] Interval Transfer Bitrate'
    if (string_index_retr in temp):
        temp2 = temp[temp.index(string_index_retr) + len(string_index_retr):]
        temp3 = temp2[temp2.index(string_index_retr) + len(string_index_retr):].strip().split('\n')[0].split(' ')
    else:
        temp2 = temp[temp.index(string_index) + len(string_index):]
        temp3 = temp2[temp2.index(string_index) + len(string_index):].strip().split('\n')[0].split(' ')
    return str(temp3[4]) + ' ' + temp3[5]

# Parse files

In [3]:
results_path = 'results/q1'
results = {}
for file_name in os.listdir(results_path):
    file_path = os.path.join(results_path, file_name)
    if ('bbr' in file_name):
        y_label = int(file_name.split('_')[-1])
        if ('client' in file_name):
            results['bbr_client_' + str(y_label)] = parse_server(file_path)
        elif ('server' in file_name):
            results['bbr_server_' + str(y_label)] = parse_server(file_path)
        elif ('ping' in file_name):
            results['bbr_ping_' + str(y_label)] = parse_ping(file_path, 350).mean()
    elif ('cubic' in file_name):
        y_label = int(file_name.split('_')[-1])
        if ('client' in file_name):
            results['cubic_client_' + str(y_label)] = parse_server(file_path)
        elif ('server' in file_name):
            results['cubic_server_' + str(y_label)] = parse_server(file_path)
        elif ('ping' in file_name):
            results['cubic_ping_' + str(y_label)] = parse_ping(file_path, 350).mean()

In [4]:
results

{'bbr_client_0': '1.91 GBytes',
 'bbr_client_1': '1.92 GBytes',
 'cubic_client_0': '1.96 GBytes',
 'cubic_client_1': '109 MBytes',
 'bbr_server_0': '1.90 GBytes',
 'bbr_server_1': '1.91 GBytes',
 'cubic_server_0': '1.95 GBytes',
 'cubic_server_1': '109 MBytes',
 'bbr_ping_0': 45.807142857142864,
 'bbr_ping_1': 43.3960843373494,
 'cubic_ping_0': 774.4499999999999,
 'cubic_ping_1': 41.013690476190476}